Python Extrac Transform Load 

In [2]:
# import mainly libraries
import requests
import numpy as np
import matplotlib.pyplot as plt
import math as mt
import pandas as pd
from sqlalchemy import create_engine

In [3]:
def extract()->dict:
  """
  This API extracts data from 
  http://universities.hipolabs.com/search?country=United+States
  """
  API_URL = "http://universities.hipolabs.com/search?country=United+States"
  data = requests.get(API_URL).json()
  return data


In [4]:
def transform(data:dict) -> pd.DataFrame:
  df = pd.DataFrame(data)
  print(f"Total number of unioversities from API {len(data)}")
  df = df[df["name"].str.contains("California")]
  print(f"Number of universities in California {len(df)}")
  df['domains'] = [','.join(map(str,l)) for l in df['domains']]
  df['web_pages'] = [','.join(map(str,l)) for l in df['web_pages']]
  df = df.reset_index(drop=True)
  return df[["domains","country","web_pages","name"]]


In [5]:
def load(df:pd.DataFrame)-> None:
  disk_engine = create_engine('sqlite:///my_lite_store.db')
  df.to_sql('cal_uni', disk_engine,if_exists='replace')

In [13]:
data  = extract()
df = transform(data)
load(df)
#print(df[['name','web_pages']].head(2).align(df,join='right'))
print(df[['name','web_pages']].head(2))

Total number of unioversities from API 2334
Number of universities in California 44
                                          name                   web_pages
0  California State University Channel Islands      https://www.csuci.edu/
1                California Baptist University  http://www.calbaptist.edu/


In [9]:
help(df.align)

Help on method align in module pandas.core.frame:

align(other, join='outer', axis=None, level=None, copy=True, fill_value=None, method=None, limit=None, fill_axis=0, broadcast_axis=None) -> 'DataFrame' method of pandas.core.frame.DataFrame instance
    Align two objects on their axes with the specified join method.
    
    Join method is specified for each axis Index.
    
    Parameters
    ----------
    other : DataFrame or Series
    join : {'outer', 'inner', 'left', 'right'}, default 'outer'
    axis : allowed axis of the other object, default None
        Align on index (0), columns (1), or both (None).
    level : int or level name, default None
        Broadcast across a level, matching Index values on the
        passed MultiIndex level.
    copy : bool, default True
        Always returns new objects. If copy=False and no reindexing is
        required then original objects are returned.
    fill_value : scalar, default np.NaN
        Value to use for missing values. Defaul